In [1]:
import numpy as np
import logging
import os
from pymoo.core.problem import ElementwiseProblem
import sys

In [2]:
sys.path.append(os.path.abspath('../src'))
from integrated_problem import *
from base_problem import TravelItineraryProblem

In [3]:
def save_solution_to_file(problem, solution, filename="log/notebook_solution.log"):
        """
        Save the solution to a log file in a readable format
        
        Args:
            problem: The problem instance
            solution: The solution vector
            filename: Path to the log file
        """
        # Create log directory if it doesn't exist
        os.makedirs(os.path.dirname(filename), exist_ok=True)
        
        # Reshape solution into x_var and u_var
        x_var = solution[:problem.x_shape].reshape(problem.NUM_DAYS, problem.num_transport_types, 
                                                problem.num_locations, problem.num_locations)
        u_var = solution[problem.x_shape:].reshape(problem.NUM_DAYS, problem.num_locations)
        
        # Open file for writing
        with open(filename, 'w') as f:
            f.write("=== HEURISTIC SOLUTION DETAILS ===\n\n")
            
            # Write solution metadata
            f.write(f"Problem dimensions: {problem.NUM_DAYS} days, {problem.num_locations} locations\n")
            f.write(f"Budget: ${problem.budget}\n")
            f.write(f"Solution vector shape: {solution.shape}, dtype: {solution.dtype}\n\n")
            
            # Write x_var (binary decision variables)
            f.write("=== BINARY DECISION VARIABLES (x_var) ===\n")
            f.write(f"Shape: {x_var.shape}\n\n")
            
            for day in range(problem.NUM_DAYS):
                f.write(f"DAY {day+1}:\n")
                
                # For each transport type
                for j, transport_type in enumerate(problem.transport_types):
                    f.write(f"  Transport mode: {transport_type}\n")
                    
                    # Create a matrix of routes
                    route_matrix = np.zeros((problem.num_locations, problem.num_locations), dtype=int)
                    for k in range(problem.num_locations):
                        for l in range(problem.num_locations):
                            route_matrix[k, l] = x_var[day, j, k, l]
                    
                    # Write route matrix with location names
                    f.write("    Origin \\ Destination")
                    for l in range(problem.num_locations):
                        name = problem.locations[l]["name"]
                        # Truncate long names
                        if len(name) > 15:
                            name = name[:12] + "..."
                        f.write(f" | {name:15s}")
                    f.write("\n")
                    
                    f.write("    " + "-" * (20 + 18 * problem.num_locations) + "\n")
                    
                    for k in range(problem.num_locations):
                        name = problem.locations[k]["name"]
                        if len(name) > 15:
                            name = name[:12] + "..."
                        f.write(f"    {name:15s} | ")
                        
                        for l in range(problem.num_locations):
                            f.write(f"{int(route_matrix[k, l]):15d} | ")
                        f.write("\n")
                    
                    f.write("\n")
                
                f.write("\n")
            
            # Write u_var (time variables)
            f.write("=== TIME VARIABLES (u_var) ===\n")
            f.write(f"Shape: {u_var.shape}\n\n")
            
            for day in range(problem.NUM_DAYS):
                f.write(f"DAY {day+1} (times in minutes since day start):\n")
                f.write("  Location ID | Location Name                  | Type       | Time\n")
                f.write("  " + "-" * 70 + "\n")
                
                # Sort by time for readability
                time_sorted_indices = np.argsort(u_var[day, :])
                
                for idx in time_sorted_indices:
                    time_val = u_var[day, idx]
                    if time_val > 0:  # Only show locations that are visited
                        location_name = problem.locations[idx]["name"]
                        location_type = problem.locations[idx]["type"]
                        
                        # Format time as HH:MM
                        hours = int(time_val // 60)
                        minutes = int(time_val % 60)
                        time_str = f"{hours:02d}:{minutes:02d}"
                        
                        f.write(f"  {idx:11d} | {location_name:30s} | {location_type:10s} | {time_str} ({time_val:.1f} min)\n")
                
                f.write("\n")
            
            # Write derived information
            f.write("=== DERIVED INFORMATION ===\n\n")
            
            # Calculate total visits
            total_visits = np.sum(x_var)
            f.write(f"Total visits: {total_visits}\n")
            
            # Count visits by location type
            attraction_visits = 0
            hawker_visits = 0
            hotel_visits = 0
            
            for i in range(problem.num_locations):
                loc_type = problem.locations[i]["type"]
                visit_count = np.sum(x_var[:, :, :, i])  # Count as destination
                
                if loc_type == "attraction":
                    attraction_visits += visit_count
                elif loc_type == "hawker":
                    hawker_visits += visit_count
                elif loc_type == "hotel":
                    hotel_visits += visit_count
            
            f.write(f"Attraction visits: {attraction_visits}\n")
            f.write(f"Hawker visits: {hawker_visits}\n")
            f.write(f"Hotel visits: {hotel_visits}\n\n")
            
            # Write raw solution vector if needed
            f.write("=== RAW SOLUTION VECTOR ===\n")
            f.write(f"First 20 elements: {solution[:20]}\n")
            f.write(f"Last 20 elements: {solution[-20:]}\n")

def print_daily_routes(problem, results):
        """
        Print the daily routes in a clear, readable format
        
        Args:
            results: The validation results dictionary
        """
        # Print feasibility status
        if results["is_feasible"]:
            print("\n✅ Solution is FEASIBLE")
        else:
            print("\n❌ Solution is INFEASIBLE")
            print(f"   - Inequality violations: {len(results['inequality_violations'])}")
            print(f"   - Equality violations: {len(results['equality_violations'])}")
        
        # Print summary
        print(f"\n📊 Summary:")
        print(f"   - Total cost: ${results['total_cost']:.2f}")
        print(f"   - Total travel time: {results['total_travel_time']:.1f} minutes")
        print(f"   - Total satisfaction: {results['total_satisfaction']:.1f}")
        
        # Print daily routes
        for day, route in enumerate(results["details"]["daily_routes"]):
            print(f"\n===== Day {day+1} Itinerary =====")
            
            prev_time = None
            
            for i, step in enumerate(route):
                location_type = step["type"]
                location_name = step["name"]
                arrival_time = step["time"]
                transport = step["transport_from_prev"]
                
                # Format time as hours:minutes
                hours = int(arrival_time // 60)
                minutes = int(arrival_time % 60)
                time_str = f"{hours:02d}:{minutes:02d}"
                
                # Skip printing transport for first location
                if i == 0:
                    print(f"{time_str} - Start at {location_name}")
                    prev_time = arrival_time
                else:
                    # Calculate travel time from previous location
                    travel_time = arrival_time - prev_time
                    
                    print(f"{time_str} - Arrive at {location_name} ({location_type}) by {transport}")
                    print(f"       - Travel time: {travel_time:.1f} minutes")
                    
                    # Calculate duration at location
                    if location_type in ["attraction", "hawker"]:
                        duration = problem.locations[step["location"]].get("duration", 60)
                        print(f"       - Stay for {duration:.1f} minutes")
                        # Update previous time to include stay duration
                        prev_time = arrival_time + duration
                    else:
                        prev_time = arrival_time
        
        # Print constraint violations if any
        if not results["is_feasible"]:
            print("\n⚠️ Constraint Violations:")
            
            if results["inequality_violations"]:
                print("\nInequality Violations:")
                for i, violation in enumerate(results["inequality_violations"]):
                    print(f"{i+1}. {violation['type']}: {violation.get('value', 'N/A')} (limit: {violation.get('limit', 'N/A')})")
            
            if results["equality_violations"]:
                print("\nEquality Violations:")
                for i, violation in enumerate(results["equality_violations"]):
                    print(f"{i+1}. {violation['type']}: {violation.get('value', 'N/A')} != {violation.get('required', 'N/A')}")

def validate_heuristic_solution(problem, solution):
        """
        Validate a solution against all constraints in the problem
        and trace daily routes without repetition
        
        Args:
            problem: The TravelItineraryProblem instance
            solution: The solution vector to validate
            
        Returns:
            dict: A dictionary with constraint violation information
        """
        # Reshape solution into x_var and u_var
        x_var = solution[:problem.x_shape].reshape(problem.NUM_DAYS, problem.num_transport_types, 
                                                problem.num_locations, problem.num_locations)
        u_var = solution[problem.x_shape:].reshape(problem.NUM_DAYS, problem.num_locations)
        
        # Initialize results
        results = {
            "is_feasible": True,
            "inequality_violations": [],
            "equality_violations": [],
            "total_cost": 0,
            "total_travel_time": 0,
            "total_satisfaction": 0,
            "details": {}
        }
        
        # Calculate solution properties
        total_cost = problem.NUM_DAYS * problem.HOTEL_COST
        total_travel_time = 0
        total_satisfaction = 0
        
        def trace_daily_routes(problem, x_var, u_var, day):
            """
            Trace the route for a specific day, avoiding cycles and repetition
            
            Args:
                problem: The problem instance
                x_var: The binary decision variables reshaped to 4D
                u_var: The time variables reshaped to 2D
                day: The day to trace
                
            Returns:
                list: Ordered sequence of locations visited
            """
            route = []
            visited = set()  # Track visited locations to avoid cycles
            current_loc = 0  # Start at hotel (assumed to be index 0)
            
            # Add starting point
            route.append({
                "location": current_loc,
                "name": problem.locations[current_loc]["name"],
                "type": problem.locations[current_loc]["type"],
                "time": float(u_var[day, current_loc]),
                "transport_from_prev": None
            })
            visited.add(current_loc)
            
            # Follow the route based on time ordering
            while True:
                next_loc = None
                next_transport = None
                min_next_time = float('inf')
                
                # Find the next location in time sequence
                for j in range(problem.num_transport_types):
                    for l in range(problem.num_locations):
                        # Check if there's a route from current location to location l
                        if x_var[day, j, current_loc, l] > 0 and l not in visited:
                            # Check if this is the earliest next location
                            if u_var[day, l] < min_next_time:
                                min_next_time = u_var[day, l]
                                next_loc = l
                                next_transport = j
                
                # If no next location found, we've completed the route
                if next_loc is None:
                    break
                
                # Add next location to route
                route.append({
                    "location": next_loc,
                    "name": problem.locations[next_loc]["name"],
                    "type": problem.locations[next_loc]["type"],
                    "time": float(u_var[day, next_loc]),
                    "transport_from_prev": problem.transport_types[next_transport]
                })
                
                # Mark location as visited and move to it
                visited.add(next_loc)
                current_loc = next_loc
                
                # Safety check to prevent infinite loops
                if len(route) > problem.num_locations:
                    break
            
            return route
        
        # ===== Check Inequality Constraints =====
        
        # 1. For attractions, visit at most once
        for k in range(problem.num_locations):
            if problem.locations[k]["type"] == "attraction":
                visits_as_source = np.sum(x_var[:, :, k, :])
                visits_as_dest = np.sum(x_var[:, :, :, k])
                
                if visits_as_source > 1:
                    results["is_feasible"] = False
                    results["inequality_violations"].append({
                        "type": "attraction_max_once_source",
                        "location": k,
                        "name": problem.locations[k]["name"],
                        "value": float(visits_as_source),
                        "limit": 1
                    })
                
                if visits_as_dest > 1:
                    results["is_feasible"] = False
                    results["inequality_violations"].append({
                        "type": "attraction_max_once_dest",
                        "location": k,
                        "name": problem.locations[k]["name"],
                        "value": float(visits_as_dest),
                        "limit": 1
                    })
        
        # 2. For hawkers, visit at most once per day
        for day in range(problem.NUM_DAYS):
            for k in range(problem.num_locations):
                if problem.locations[k]["type"] == "hawker":
                    daily_visits_source = np.sum(x_var[day, :, k, :])
                    daily_visits_dest = np.sum(x_var[day, :, :, k])
                    
                    if daily_visits_source > 1:
                        results["is_feasible"] = False
                        results["inequality_violations"].append({
                            "type": "hawker_max_once_daily_source",
                            "day": day,
                            "location": k,
                            "name": problem.locations[k]["name"],
                            "value": float(daily_visits_source),
                            "limit": 1
                        })
                    
                    if daily_visits_dest > 1:
                        results["is_feasible"] = False
                        results["inequality_violations"].append({
                            "type": "hawker_max_once_daily_dest",
                            "day": day,
                            "location": k,
                            "name": problem.locations[k]["name"],
                            "value": float(daily_visits_dest),
                            "limit": 1
                        })
        
        # 3. Check start time constraint (must start from hotel at START_TIME)
        for day in range(problem.NUM_DAYS):
            if u_var[day, 0] < problem.START_TIME:
                results["is_feasible"] = False
                results["inequality_violations"].append({
                    "type": "start_time_before_minimum",
                    "day": day,
                    "value": float(u_var[day, 0]),
                    "limit": problem.START_TIME
                })
        
        # 4. Check time constraints
        for day in range(problem.NUM_DAYS):
            for j in range(problem.num_transport_types):
                for k in range(problem.num_locations):
                    for l in range(1, problem.num_locations):
                        if k == l:
                            continue
                        
                        # Skip if route not chosen
                        if x_var[day, j, k, l] == 0:
                            continue
                        
                        # Check time constraints
                        time_finish_l = u_var[day, l]
                        transport_hour = problem.get_transport_hour(u_var[day, k])
                        
                        try:
                            transport_value = problem.transport_matrix[(problem.locations[k]["name"], 
                                                                    problem.locations[l]["name"], 
                                                                    transport_hour)][problem.transport_types[j]]
                            
                            time_should_finish_l = u_var[day, k] + transport_value["duration"] + problem.locations[l]["duration"]
                            
                            if time_finish_l < time_should_finish_l:
                                results["is_feasible"] = False
                                results["inequality_violations"].append({
                                    "type": "time_constraint_violated",
                                    "day": day,
                                    "transport": j,
                                    "from": k,
                                    "to": l,
                                    "from_name": problem.locations[k]["name"],
                                    "to_name": problem.locations[l]["name"],
                                    "actual_finish": float(time_finish_l),
                                    "required_finish": float(time_should_finish_l)
                                })
                        except KeyError:
                            results["inequality_violations"].append({
                                "type": "missing_transport_data",
                                "day": day,
                                "transport": j,
                                "from": k,
                                "to": l,
                                "from_name": problem.locations[k]["name"],
                                "to_name": problem.locations[l]["name"],
                                "hour": transport_hour
                            })
        
        # 5. Check transport type constraints (can't use multiple transport types for same route)
        for day in range(problem.NUM_DAYS):
            for k in range(problem.num_locations):
                for l in range(problem.num_locations):
                    transport_sum = np.sum(x_var[day, :, k, l])
                    
                    if transport_sum > 1:
                        results["is_feasible"] = False
                        results["inequality_violations"].append({
                            "type": "multiple_transport_types",
                            "day": day,
                            "from": k,
                            "to": l,
                            "from_name": problem.locations[k]["name"],
                            "to_name": problem.locations[l]["name"],
                            "value": float(transport_sum),
                            "limit": 1
                        })
        
        # 6. Check lunch and dinner constraints
        for day in range(problem.NUM_DAYS):
            lunch_visits = 0
            dinner_visits = 0
            hawker_sum = 0
            
            for k in range(problem.num_locations):
                if problem.locations[k]["type"] == "hawker":
                    hawker_sum += np.sum(x_var[day, :, k, :])
                    
                    # Check if this hawker visit is during lunch or dinner
                    if u_var[day, k] >= problem.LUNCH_START and u_var[day, k] <= problem.LUNCH_END:
                        lunch_visits += 1
                    
                    if u_var[day, k] >= problem.DINNER_START and u_var[day, k] <= problem.DINNER_END:
                        dinner_visits += 1
            
            # Every day, must go to hawkers at least twice
            if hawker_sum < 2:
                results["is_feasible"] = False
                results["inequality_violations"].append({
                    "type": "insufficient_hawker_visits",
                    "day": day,
                    "value": float(hawker_sum),
                    "limit": 2
                })
            
            # Must visit a hawker during lunch time
            if lunch_visits < 1:
                results["is_feasible"] = False
                results["inequality_violations"].append({
                    "type": "no_lunch_hawker_visit",
                    "day": day,
                    "value": lunch_visits,
                    "limit": 1
                })
            
            # Must visit a hawker during dinner time
            if dinner_visits < 1:
                results["is_feasible"] = False
                results["inequality_violations"].append({
                    "type": "no_dinner_hawker_visit",
                    "day": day,
                    "value": dinner_visits,
                    "limit": 1
                })
        
        # 7. Calculate total cost
        for day in range(problem.NUM_DAYS):
            for j in range(problem.num_transport_types):
                for k in range(problem.num_locations):
                    for l in range(problem.num_locations):
                        if k == l:
                            continue
                        
                        if x_var[day, j, k, l] == 1:
                            try:
                                transport_hour = problem.get_transport_hour(u_var[day, k])
                                transport_value = problem.transport_matrix[(problem.locations[k]["name"], 
                                                                        problem.locations[l]["name"], 
                                                                        transport_hour)][problem.transport_types[j]]
                                
                                total_travel_time += transport_value["duration"]
                                total_cost += transport_value["price"]
                                
                                # Add location costs
                                if problem.locations[l]["type"] == "attraction":
                                    total_cost += problem.locations[l]["entrance_fee"]
                                    total_satisfaction += problem.locations[l]["satisfaction"]
                                elif problem.locations[l]["type"] == "hawker":
                                    total_cost += 10  # Assumed meal cost
                                    total_satisfaction += problem.locations[l]["rating"]
                            except KeyError:
                                # Missing transport data
                                pass
        
        # Check budget constraint
        if total_cost > problem.budget:
            results["is_feasible"] = False
            results["inequality_violations"].append({
                "type": "budget_exceeded",
                "value": float(total_cost),
                "limit": problem.budget
            })
        
        # 8. Check minimum and maximum visits
        min_visits = problem.NUM_DAYS * 2  # Minimum 2 visits per day (lunch & dinner)
        max_visits = problem.NUM_DAYS * 6  # Maximum reasonable visits per day
        total_visits = np.sum(x_var)
        
        if total_visits < min_visits:
            results["is_feasible"] = False
            results["inequality_violations"].append({
                "type": "insufficient_total_visits",
                "value": float(total_visits),
                "limit": min_visits
            })
        
        if total_visits > max_visits:
            results["is_feasible"] = False
            results["inequality_violations"].append({
                "type": "excessive_total_visits",
                "value": float(total_visits),
                "limit": max_visits
            })
        
        # ===== Check Equality Constraints =====
        
        # 1. For each day, hotel must be the starting point
        for day in range(problem.NUM_DAYS):
            # Hotel index is 0
            hotel_outgoing = np.sum(x_var[day, :, 0, :])
            if hotel_outgoing != 1:
                results["is_feasible"] = False
                results["equality_violations"].append({
                    "type": "hotel_not_starting_point",
                    "day": day,
                    "value": float(hotel_outgoing),
                    "required": 1
                })
        
        # 2. For each day, must return to hotel
        for day in range(problem.NUM_DAYS):
            # Find last location
            last_time = np.max(u_var[day, :])
            last_locations = np.where(u_var[day, :] == last_time)[0]
            
            # If last location is not hotel (0), check if there's a route back to hotel
            if len(last_locations) > 0 and last_locations[0] != 0:
                last_loc = last_locations[0]
                returns_to_hotel = np.sum(x_var[day, :, last_loc, 0])
                
                if returns_to_hotel != 1:
                    results["is_feasible"] = False
                    results["equality_violations"].append({
                        "type": "not_returning_to_hotel",
                        "day": day,
                        "last_location": last_loc,
                        "last_location_name": problem.locations[last_loc]["name"],
                        "value": float(returns_to_hotel),
                        "required": 1
                    })
        
        # 3. Flow conservation (in = out)
        for day in range(problem.NUM_DAYS):
            for k in range(problem.num_locations):
                incoming = np.sum(x_var[day, :, :, k])
                outgoing = np.sum(x_var[day, :, k, :])
                
                if incoming != outgoing:
                    results["is_feasible"] = False
                    results["equality_violations"].append({
                        "type": "flow_conservation_violated",
                        "day": day,
                        "location": k,
                        "location_name": problem.locations[k]["name"],
                        "incoming": float(incoming),
                        "outgoing": float(outgoing)
                    })
        
        # 4. If attraction is visited as source, it must be visited as destination
        for k in range(problem.num_locations):
            if problem.locations[k]["type"] == "attraction":
                as_source = np.sum(x_var[:, :, k, :])
                as_dest = np.sum(x_var[:, :, :, k])
                
                if as_source != as_dest:
                    results["is_feasible"] = False
                    results["equality_violations"].append({
                        "type": "attraction_source_dest_mismatch",
                        "location": k,
                        "name": problem.locations[k]["name"],
                        "as_source": float(as_source),
                        "as_dest": float(as_dest)
                    })
        
        # Store totals
        results["total_cost"] = float(total_cost)
        results["total_travel_time"] = float(total_travel_time)
        results["total_satisfaction"] = float(total_satisfaction)
        
        # ===== Generate non-repetitive daily routes =====
        
        results["details"] = {
            "visited_attractions": [problem.locations[i]["name"] for i in range(problem.num_locations) 
                                if problem.locations[i]["type"] == "attraction" and np.sum(x_var[:, :, i, :]) > 0],
            "daily_routes": [],
            "daily_timelines": []
        }
        
        # Generate daily routes using improved tracing
        for day in range(problem.NUM_DAYS):
            route = trace_daily_routes(problem, x_var, u_var, day)
            results["details"]["daily_routes"].append(route)
            
            # Also create a simpler timeline for easy viewing
            timeline = []
            for step in route:
                timeline.append({
                    "location": step["location"],
                    "name": step["name"],
                    "type": step["type"],
                    "time": step["time"],
                    "transport": step["transport_from_prev"]
                })
            
            results["details"]["daily_timelines"].append(timeline)
        
        return results

In [4]:
import json

def get_transport_matrix():
    BASE_PATH = "../data/routeData/"
    daytimes = [("morning", 8), ("midday", 12), ("evening", 16), ("night", 20)]
    transport_matrix = {}

    for time_in_day, hour in daytimes:
        filepath = BASE_PATH + f"route_matrix_{time_in_day}.json"

        with open(filepath, 'r') as f:
            route_matrix = json.load(f)

        for route in route_matrix["routes"]:
            this_route = route_matrix["routes"][route]
            origin = this_route["origin_name"]
            destination = this_route["destination_name"]
            transport_matrix[(origin, destination, hour)] = {
                "transit": {
                    "duration": int(this_route["transit"]["duration_minutes"]),
                    "price": int(this_route["transit"]["fare_sgd"]),
                },
                "drive": {
                    "duration": int(this_route["drive"]["duration_minutes"]),
                    "price": int(this_route["drive"]["fare_sgd"]),
                }
            }
    
    return transport_matrix


def get_all_locations():
    BASE_PATH = "../data/routeData/"

    with open(BASE_PATH + "route_matrix_morning.json", 'r') as f:
        route_matrix = json.load(f)
    
    locations = [route_matrix["locations"][location_id] for location_id in route_matrix["locations"]]
    return locations

In [5]:
import numpy as np

# Load the .npy file
file_path = "../results/best_solution.npy"  # Change this to your file path
best_solution = np.load(file_path)

# Load the .npy file
file_path = "../results/best_objectives.npy"  # Change this to your file path
F = np.load(file_path)

# Print basic information about the loaded data
print(f"Data shape: {best_solution.shape}")
print(f"Data type: {best_solution.dtype}")
print(f"First few elements: {best_solution[:10]}")  # Print first 10 elements

Data shape: (200, 1806)
Data type: int64
First few elements: [[-1  0  0 ...  0  0  0]
 [-1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [-1  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]


In [6]:
weights = np.array([0.3, 0.3, 0.4])

In [7]:
from pymoo.decomposition.asf import ASF

decomp = ASF()

approx_ideal = F.min(axis=0)
approx_nadir = F.max(axis=0)

nF = (F - approx_ideal) / (approx_nadir - approx_ideal)

i = decomp.do(nF, 1/weights).argmin()

print("Best regarding ASF: Point \ni = %s\nF = %s" % (i, F[i]))

pymoo_solution = best_solution[i]

Best regarding ASF: Point 
i = 0
F = [259.77196607  84.6        -38.66869429]


C:\Users\danie\AppData\Local\Temp\ipykernel_75708\1228327285.py:8: RuntimeWarning: invalid value encountered in divide
  nF = (F - approx_ideal) / (approx_nadir - approx_ideal)


In [8]:
file_path = "../results/init_solution.npy"  # Change this to your file path
init_solution = np.load(file_path)

# Print basic information about the loaded data
print(f"Data shape: {init_solution.shape}")
print(f"Data type: {init_solution.dtype}")
print(f"First few elements: {init_solution[:10]}")  # Print first 10 elements

Data shape: (1806,)
Data type: float64
First few elements: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [11]:
hotel_name=None 
budget=1000
num_days=2
max_attractions=12
max_hawkers=8
# Define constraints
inequality_constraints = []
    # """
    # day_one_attraction_limit = np.sum(x_var[0, :, :, :]) - 3 # should be <= 3
    # out["G"].append(day_one_attraction_limit)
    # """
# ]
equality_constraints = []
    # """out["H"].append(np.sum(x_var) - 5) # should be == 5""",
# ]
# Get user-specified hotel or use default

hotel = get_hotel_waypoint(hotel_name if hotel_name else None)
# Get transport matrix and locations
logger.info("Loading transport matrix and locations...")
all_locations = get_all_locations()
transport_matrix = get_transport_matrix()

# Filter locations based on max_attractions and max_hawkers
if max_attractions is not None or max_hawkers is not None:
    logger.info(f"Filtering locations (max attractions: {max_attractions}, max hawkers: {max_hawkers})...")
    all_locations = filter_locations(all_locations, max_attractions, max_hawkers)
    logger.info(f"Filtered to {len(all_locations)} locations")

# Integrate hotel with locations and transport matrix
updated_locations, updated_matrix = integrate_hotel_with_locations(
    hotel, all_locations, transport_matrix
)
# For all locations, get necessary data (only if not already present)
for loc in updated_locations:
    if loc["type"] == "hawker":
        if not "rating" in loc:
            loc["rating"] = np.random.uniform(0, 5)
        if not "avg_food_price" in loc:
            loc["avg_food_price"] = np.random.uniform(5, 15)
        if not "duration" in loc:
            loc["duration"] = 60  # just standardize 60 mins
    elif loc["type"] == "attraction":
        if not "satisfaction" in loc:
            loc["satisfaction"] = np.random.uniform(0, 10)
        if not "entrance_fee" in loc:
            loc["entrance_fee"] = np.random.uniform(5, 100)
        if not "duration" in loc:
            loc["duration"] = np.random.randint(30, 90)
    elif loc["type"] == "hotel":
        # Set hotel duration to 0 (no time spent at hotel for activities)
        loc["duration"] = 0
# make the problemset and solve it.
logger.info("Creating and solving the optimization problem...")
problem = TravelItineraryProblem(
    num_days=num_days,
    budget=budget,
    locations=updated_locations,
    transport_matrix=updated_matrix,
)

2025-03-21 15:50:06,132 - WARNING - No hotel information provided, using default hotel
2025-03-21 15:50:06,133 - INFO - Loading transport matrix and locations...
2025-03-21 15:50:06,151 - INFO - Filtering locations (max attractions: 12, max hawkers: 8)...
2025-03-21 15:50:06,152 - INFO - Filtered to 20 locations
2025-03-21 15:50:06,153 - INFO - Added new hotel: DEFAULT HOTEL
2025-03-21 15:50:06,154 - WARNING - Missing routes between hotel and location 'Marina Bay Sands SkyPark Observation Deck' in cached data
2025-03-21 15:50:06,155 - WARNING - Memory-cached routes for DEFAULT HOTEL don't match current locations, cache invalid
2025-03-21 15:50:06,157 - INFO - API queries_quota: 60
2025-03-21 15:50:06,158 - INFO - Computing route matrices for hotel 'DEFAULT HOTEL' with 20 destinations
2025-03-21 15:50:06,158 - INFO - Computing transit route matrix for hour 8...
2025-03-21 15:50:27,518 - INFO - Computing driving route matrix for hour 8...
2025-03-21 15:50:48,670 - INFO - Computing transi

In [12]:
from utils.export_itinerary import export_itinerary
import os
itinerary_file = export_itinerary(problem, pymoo_solution, "results/singapore_itinerary.txt")

In [14]:
# This returns a boolean array with element-wise comparisons
comparison = init_solution == pymoo_solution
print(comparison)  # [True True True]

# To check if ALL elements are equal
are_equal = np.all(init_solution == pymoo_solution)
print(are_equal)  # True

[False  True  True ...  True  True  True]
False


In [17]:
init_solution_int = init_solution.astype(int)

In [ ]:
init_solution_int